* handle baby funcs and user funcs
* sometimes v is a Value sometimes it's just given


* VectorizeUserFunc? What?
* wtf is up with Filter
* Reduce vs ReduceSeq

Ops:
    - Get = ._
    - Value : val, type
    VectorizeUserFun : function, arg
    Transpose : arg
    Join : arg
    Zip : arg, arg
    Map : function, arg
    Filter : ?
    ReduceSeq : ?
    Reduce : function, val, arg
    Lambda : fun(...) | \(...)
    - Params
    UserFuncs

* strip quotes from function?
* what's the point of \?
* handle getting el from tuple (Get)
* :>> ??? = pipe in to next op I think
* 0 vs 0.0f vs "0.0f" vs 1 vs 3.40282347e+38

Ops:
    - Get = ._
    - Value : val, type
    VectorizeUserFun : function, arg
    Transpose : arg
    Join : arg
    Zip : arg, arg
    Map : function, arg
    Filter : ?
    ReduceSeq : ?
    Reduce : function, val, arg
    Lambda : fun(...) | \(...)
    - Params
    UserFuncs

Piping:
    o : apply - f o g(x) = f(g(x))
    \$ : piping  - f \$ x = f(x)
    :>> : reverse piping - x :>> f = f(x)

Types:
 'TupleType',
 'Float4',
 'Int',
 'Float',
 'ArrayType',

In [1]:
import os, re

def flatten(lol):
    return [val for sublist in lol for val in sublist]
def printls(ls):
    for item in ls:
        print(item)
        
def getcn(thingy):
    return thingy.__class__.__name__
liftdir = "/Users/hebe/Dropbox/Yale-NUS/Yr4/capstone/forrealsies/lift"

program_names = os.listdir(liftdir + "/highLevel")
program_names = [x for x in program_names if 
            os.path.isfile(os.path.join(liftdir,"highLevel",x)) and ('.' not in x)]

In [2]:
programs = []

for program_name in program_names:
    with open(os.path.join(liftdir,"highLevel",program_name)) as f:
        programs.append(f.read())

In [3]:
# Input: string
# Output: tuple: (string before brackets, 
#                 top-level contents of first brackets (or full string if no brackets),
#                 string after brackets)
# Error state: If brackets not closed
def inbc(s):
    if '(' not in s:
        return "",s,""
    start = s.index('(')
    open_ctr = 0
    for i in range(start+1, len(s)):
        if s[i] == '(':
            open_ctr += 1
        if s[i] == ')':
            if open_ctr == 0:
                return s[0:start], s[start+1:i], s[i+1:]
            else:
                open_ctr -= 1
    return False

def getcbc(s):
    if '{' not in s:
        return s
    start = s.index('{')
    open_ctr = 0
    for i in range(start+1, len(s)):
        if s[i] == '{':
            open_ctr += 1
        if s[i] == '}':
            if open_ctr == 0:
                return s[start+1:i]
            else:
                open_ctr -= 1
    return False

def getbc(s):
    return inbc(s)[1]

def inbc_rev(s):
    if '(' not in s:
        return "",s,""
    end = s.rindex(')')
    open_ctr = 0
    for i in range(end-1, -1, -1):
        if s[i] == ')':
            open_ctr += 1
        if s[i] == '(':
            if open_ctr == 0:
                return s[0:i], s[i+1:end], s[end+1:]
            else:
                open_ctr -= 1
    return False    

# Input: Overarching program
# Output: Main function
def getfunc(s):
    start = s.index('\nfun')
    s = getbc(s[start:])
    return re.sub(r"([a-zA-Z0-9]+)._([0-9]+)", r"Get(\g<1>, \g<2>)",s)

# Input: Overarching program
# Output: List of all SizeVar names
def getsvs(s):
    return re.findall(r'SizeVar\("([a-zA-Z]+)"\)', s)

# Input: function
# Output: body of function
def getbody(s):
    i = s.index("=>")
    s = s[i+2:].strip()
    if s[0] == '(':
        s = getbc(s)
    if s[0] == '{':
        s = getcbc(s)
    if s[0] == '(':
        s = getbc(s)
    return s
def getmainbody(s):
    s = getfunc(s)
    return getbody(s)

In [4]:
# Input: string of comma-separated items
# Output: list of top-level items
def commasplit(s):
    commas = [-1]
    open_ctr = 0
    for i in range(len(s)):
        if s[i] in ['(', '{', '[']:
            open_ctr += 1
        if s[i] in [')', '}', ']']:
            open_ctr -= 1
        if s[i] == ',' and open_ctr == 0:
            commas.append(i)
    commas.append(len(s))
    broken = [s[commas[j-1]+1:commas[j]] for j in range(1, len(commas))]
    return broken

# Input: string of comma-separated items
# Output: list of top-level items
def commasplit_rev(s):
    commas = [len(s)]
    open_ctr = 0
    for i in range(len(s)-1 , -1, -1):
        if s[i] in [')', '}', ']']:
            open_ctr += 1
        if s[i] in ['(', '{', '[']:
            open_ctr -= 1
        if s[i] == ',' and open_ctr == 0:
            commas.append(i)
    commas.append(-1)
    commas.reverse()
    broken = [s[commas[j-1]+1:commas[j]] for j in range(1, len(commas))]
    return [x for x in broken if len(x) > 0]

# Input: Main function with input types, input names, and function
# Output: list of pairs (input name, input type)
def getinputs(s):
    f = s.split("=>")[0]
    f = commasplit_rev(f)
    t = f[0:-1]
    n = getbc(f[-1])
    n = commasplit_rev(n)
    return [(a.strip(), b.strip()) for a, b in zip(n,t)]
def getmaininputs(s):
    return getinputs(getfunc(s))


# Input: lambda function
# Output: names of paramaters
def getnames(s):
    f = s.split("=>")[0]
    ns = getbc(f)
    return commasplit(ns)


# Input: Overarching program
# Output: Given subfunctions used in main function
def getsubfuncs(s):
    sfnames = re.findall(r"\ndef +([a-zA-Z0-9]+) *= *fun", s)
    return [(name, getbc(s[s.index(name):])) for name in sfnames]
def getsplit(s):
    return [x for x in re.split(r"[\(\){},\s]", s) if len(x) > 0]

def getNextOp(s):
    s = s.strip()
    return re.match(r"([A-Za-z0-9]+)", s).groups()[0]

def getNextParams(s):
    return commasplit_rev(getbc(s))
def hasparams(s):
    return '(' in s or '{' in s

def isfunction(s):
    if s.strip().startswith(("\(", "fun(")):
        return True
    return False

# Input: body of function
# Output: list of top-level functions separated by operators
def componentsplit(s):
    commas = [0]
    open_ctr = 0
    for i in range(len(s)):
        if s[i] in ['(', '{', '[']:
            open_ctr += 1
        if s[i] in [')', '}', ']']:
            open_ctr -= 1
        if open_ctr == 0:
            if s[i:].startswith("$"):
                commas += [i, i + 1]
            if s[i:].startswith(":>>"):
                commas += [i, i + 3]
            if re.match(r".\bo\b", s[i:]):
                commas += [i+1, i + 2]
    commas.append(len(s))
#     print(commas)
    broken = [s[commas[j-1]:commas[j]].strip() for j in range(1, len(commas))]
    return broken

In [5]:
class Channel:
    def __init__ (self, name, src, dst, init_token = 0):
        self.name = name
        self.src_port = src
        self.src_act = src.split("_")[0]
        self.dst_port = dst
        self.dst_act = dst.split("_")[0]
        self.init_token = init_token
        self.size = 1
    def __str__(self):
        return "<channel name='"+self.name+"' srcActor='"+self.src_act+"' srcPort='"+self.src_port+"' dstActor='"+self.dst_act+"' dstPort='"+self.dst_port+"' size='"+str(self.size)+"' initialTokens='"+str(self.init_token)+"'/>"


class Port:
    def __init__ (self, direction, name, rate):
        self.name = name
        self.direction = direction
        self.rate = rate
    def __str__(self):
        return "<port type='"+self.direction+"' name='"+self.name+"' rate='"+','.join(map(str,self.rate))+"'/>"

# INPUT: list of dicts of ports - [{'dst' : Port, 'src' : Port}]
# OUTPUT: list of channels - [Channel]
def get_channels(ls):
    return [Channel(x["src"].name+"_to_"+x["dst"].name, x["src"].name, x["dst"].name) for x in ls]
    
class Param:
    def __init__(self, name, label):
        self.name = name
        self.label = label
        self.input = Port('in', name + '_in', [1])
        self.out_ctr = 0
        self.output = []
    def new_outport(self):
        np = Port('out', self.name + "x"+str(self.out_ctr)+ '_out', [1])
        self.output.append(np)
        self.out_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Param' >\n" + '\n'.join(map(str, [self.input]+self.output)) + "\n</actor>"

class Get:
    def __init__(self, name, idx):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
        self.idx = idx
    def __str__(self):
        return "<actor name='"+self.name+"' type='Get' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

class Transpose:
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Transpose' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

class Join:
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Join' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    
# Must have new value for every attachment
class Value:
    def __init__(self, name, value, kind):
        self.name = name
        self.value = value
        self.kind = kind
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Value' value='"+self.value+"' kind='"+self.kind+"'>\n" + '\n'.join(map(str, self.output)) + "\n</actor>"

class Reduce:
    def __init__(self, name, subfunc, initval, seq = False):
        self.name = name
        self.type =  "ReduceSeq" if seq else "Reduce"
        self.subfunc = subfunc
        self.initval = initval
        self.seq = seq
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='"+self.type+"'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"


class Map:
    def __init__(self, name, subfunc):
        self.name = name
        self.subfunc = subfunc
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Map'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"
    
class Zip:
    def __init__(self, name : str):
        self.name = name
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Zip'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"
class Add:
    def __init__(self, name : str):
        self.name = name
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Add'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"
class Mult:
    def __init__(self, name : str):
        self.name = name
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Mult'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

class UserFun:
    def __init__(self, name : str, label):
        self.name = name
        self.label = label
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='UserFun'  name='"+self.label+"'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"


In [6]:
def unpack_program(program):
    node_ctr = 0
    def getname():
        nonlocal node_ctr
        node_ctr += 1
        return "n" + str(node_ctr)
    
    named_nodes = {}
    
    starting_inputs = getmaininputs(program)
    sizevars = getsvs(program)
    main_func = commasplit(getfunc(program))[-1]
    
    def mk_bby_func(bby, n_inputs):
        assert(len(set(bby.strip()).intersection(set("{}()[],. "))) == 0)
        new_node = UserFun(getname(), bby)
        for i in range(n_inputs):
            new_node.new_inport()
        return {
            "nodes" : [new_node],
            "channels" : [],
            "inputs" : new_node.input,
            "output" : new_node.output
        }
        
    
    # Output: {nodes : [Node], channels : [{Port, Port}], inputs : [Port], output : Port}
    def unpack_func(func):
        nodes = []
        channels = []
        myinputs = []
        local_names = getnames(func) if "=>" in func else []

        for name in local_names:
            new_node = Param(getname(), name)
            nodes.append(new_node)
            myinputs.append(new_node.input)
            assert(name not in named_nodes)
            named_nodes[name] = new_node  
        
        # Input: body
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_body(body):
            comps = componentsplit(body)
            assert(len(comps) > 0)
            assert(len(comps) % 2 == 1)
            
            curr = unpack_comp(comps[0])
            for i in range(1, len(comps), 2):
                assert(comps[i] in ["o", "$", ":>>"])
                new = unpack_comp(comps[i+1])
                if comps[i] == ":>>":
                    assert(new["in"] and curr["out"])
                    channels.append({"dst":new["in"], "src" : curr["out"]})
                    curr["out"] = new.get("out")
                else:
                    assert(curr["in"] and new["out"])
                    channels.append({"dst" : curr["in"], "src" : new["out"]})
                    curr["in"] = new.get("in")
            return curr
            
        
        # Input: single component
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_comp(comp):
            assert(len(componentsplit(comp)) == 1)
            op = getNextOp(comp)
            if op in named_nodes.keys():
                assert(op == comp.strip())
                src = named_nodes[op]
                src_port = src.new_outport()
                return({"out" : src_port})
            if op == "Value":
                args = getNextParams(comp)
                # It's the final node
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Value(getname(), args[0], args[1])
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Get":
                args = getNextParams(comp)
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Get(getname(), args[1])
                incoming = unpack_body(args[0])["out"]
                channels.append({"src" : incoming, "dst" : new_node.input})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Transpose":
                assert(len(getNextParams(comp)) == 0)
                new_node = Transpose(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Join":
                assert(len(getNextParams(comp)) == 0)
                new_node = Join(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Zip":
                args = getNextParams(comp)
                new_node = Zip(getname())
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "add":
                args = getNextParams(comp)
                new_node = Add(getname())
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "mult":
                args = getNextParams(comp)
                new_node = Mult(getname())
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Map":
                args = getNextParams(comp)
                assert(len(args) == 1)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 1)
                new_node = Map(getname(), child_func)
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op in ["Reduce", "ReduceSeq"]:
                args = getNextParams(comp)
                assert(len(args) == 2)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 2)
                init_val = args[1]
                new_node = Reduce(getname(), child_func, init_val, op == "ReduceSeq")
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            else:
                new_node = UserFun(getname(), op)
                if hasparams(comp):
                    for arg in getNextParams(comp):
                        new_in = new_node.new_inport()
                        new_out = unpack_body(arg)["out"]
                        channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
        
        
        mybod = getbody(func) if "=>" in func else func
        res = unpack_body(mybod)
        if "=>" in func:
            assert(not res.get("in"))
        else:
            myinputs = [res["in"]]
        myoutput = res["out"]
        
        for name in local_names:
            named_nodes.pop(name)
        
        return {
            "nodes" : nodes,
            "channels" : get_channels(channels),
            "inputs" : myinputs,
            "output" : myoutput
        }
            
    parsed = unpack_func(main_func)
    return {
        "sizevars" : sizevars,
        "inputs" : starting_inputs,
        "code" : getfunc(program),
        "graph" : parsed
    }       

In [7]:

def printgraph(g):
    n = g["nodes"]
    c = g["channels"]
    printls(n)
    printls(c)
    
    for node in n:
        if getcn(node).startswith(("Reduce", "Map", "Iterate")):
            print("\n       Subgraph of", getcn(node),"node ", node.name)
            printgraph(node.subfunc)

In [8]:
parsed_programs = [unpack_program(p) for p in programs]


In [9]:
p = parsed_programs[2]
print(p["code"])


  ArrayType(Float, N),
  input =>
    Reduce(add, 0.0f) o Map(abs) $ input



In [10]:
printgraph(p["graph"])

<actor name='n1' type='Param' >
<port type='in' name='n1_in' rate='1'/>
</actor>
<actor name='n3' type='Reduce'>
<port type='out' name='n3_out' rate='1'/>
<port type='in' name='n3_in' rate='1'/>
</actor>
<actor name='n5' type='Map'>
<port type='out' name='n5_out' rate='1'/>
<port type='in' name='n5_in' rate='1'/>
</actor>
<actor name='n6' type='UserFun'  name='input'>
<port type='out' name='n6_out' rate='1'/>
</actor>
<channel name='n5_out_to_n3_in' srcActor='n5' srcPort='n5_out' dstActor='n3' dstPort='n3_in' size='1' initialTokens='0'/>
<channel name='n6_out_to_n5_in' srcActor='n6' srcPort='n6_out' dstActor='n5' dstPort='n5_in' size='1' initialTokens='0'/>

       Subgraph of Reduce node  n3
<actor name='n2' type='UserFun'  name='add'>
<port type='out' name='n2_out' rate='1'/>
<port type='in' name='n2x0_in' rate='1'/>
<port type='in' name='n2x1_in' rate='1'/>
</actor>

       Subgraph of Map node  n5
<actor name='n4' type='UserFun'  name='abs'>
<port type='out' name='n4_out' rate='1'/